In [76]:
import os
import sys
import subprocess
import json

# az account get-access-token --resource https://cognitiveservices.azure.com -o json | jq -r .accessToken

url = 'https://cognitiveservices.azure.com'
args = ['az', 'account', 'get-access-token', '--resource', url, '-o', 'json']
p = subprocess.run(args, check=True, capture_output=True)
p_stdout = p.stdout.decode()
p_stdout_dict = json.loads(p_stdout)
aad_token = p_stdout_dict["accessToken"]

# p, aad_token
# os.environ['aad_token'] = aad_token

In [ ]:
aad_token

In [78]:
prompts = [
"""Generate a summary of the below conversation in the following format:
Customer problem:
Outcome of the conversation:
Action items for follow-up:
Customer budget:
Departure city:
Destination city:

Conversation:
User: Hi there, I’m off between August 25 and September 11. I saved up 4000 for a nice trip. If I flew out from San Francisco, what are your suggestions for where I can go?
Agent: For that budget you could travel to cities in the US, Mexico, Brazil, Italy or Japan. Any preferences?
User: Excellent, I’ve always wanted to see Japan. What kind of hotel can I expect?
Agent: Great, let me check what I have. First, can I just confirm with you that this is a trip for one adult?
User: Yes it is
Agent: Great, thank you, In that case I can offer you 15 days at HOTEL Sugoi, a 3 star hotel close to a Palace. You would be staying there between August 25th and September 7th. They offer free wifi and have an excellent guest rating of 8.49/10. The entire package costs 2024.25USD. Should I book this for you?
User: That sounds really good actually. Please book me at Sugoi.
Agent: I can do that for you! Can I help you with anything else today?
User: No, thanks! Please just send me the itinerary to my email soon.

Summary:""",
"""Below is an extract from the annual financial report of a company. Extract key financial number (if present), key internal risk factors, and key external risk factors.

# Start of Report
Revenue increased $7.5 billion or 16%. Commercial products and cloud services revenue increased $4.0 billion or 13%. O365 Commercial revenue grew 22% driven by seat growth of 17% and higher revenue per user. Office Consumer products and cloud services revenue increased $474 million or 10% driven by Consumer subscription revenue, on a strong prior year comparable that benefited from transactional strength in Japan. Gross margin increased $6.5 billion or 18% driven by the change in estimated useful lives of our server and network equipment. 
Our competitors range in size from diversified global companies with significant research and development resources to small, specialized firms whose narrower product lines may let them be more effective in deploying technical, marketing, and financial resources. Barriers to entry in many of our businesses are low and many of the areas in which we compete evolve rapidly with changing and disruptive technologies, shifting user needs, and frequent introductions of new products and services. Our ability to remain competitive depends on our success in making innovative products, devices, and services that appeal to businesses and consumers.
# End of Report""",
"Once upon a time",
]

In [81]:
import requests
import json

s = requests.session()
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % aad_token
  }
url = "https://mazuseanclabopenai02.openai.azure.com/openai/deployments/text-davinci-003/completions"
params = {
    # "api_base": "https://mazuseanclabopenai02.openai.azure.com",
    # "api_type": "azure",
    # "api_version": "2022-12-01"
    "api-version": "2023-03-15-preview"
    }
data = {
  'prompt': prompts[1],
  "max_tokens": 350,
  "temperature": 0.3,
  "frequency_penalty": 0,
  "presence_penalty": 0,
  "top_p": 1,
  "best_of": 1,
  "stop": None,
}

resp = s.post(url, headers=headers, params=params, json=data)


In [82]:
resp_j = resp.json()
resp.status_code, resp_j 
# resp_j['choices'][0]['text']

(200,
 {'id': 'cmpl-7eTtwWW1wJAzSRXpSIOFus7y530Tv',
  'object': 'text_completion',
  'created': 1689882416,
  'model': 'text-davinci-003',
  'choices': [{'text': '\n\nKey Financial Number: \n- Revenue increased $7.5 billion or 16%\n- Gross margin increased $6.5 billion or 18%\n\nKey Internal Risk Factors:\n- Ability to remain competitive depends on success in making innovative products, devices, and services\n\nKey External Risk Factors:\n- Competitors range in size from diversified global companies to small, specialized firms\n- Low barriers to entry in many of our businesses\n- Rapidly changing and disruptive technologies, shifting user needs, and frequent introductions of new products and services',
    'index': 0,
    'finish_reason': 'stop',
    'logprobs': None}],
  'usage': {'completion_tokens': 114,
   'prompt_tokens': 261,
   'total_tokens': 375}})